# Batch Deep Learning Training for All Brazilian State Capitals (Stacked GRU)

This notebook trains the best deep learning model configuration (Stacked GRU, as determined in previous experiments) for each Brazilian state capital. Results are saved in a structured way for further analysis and comparison.

In [3]:
print(f"{city_name} - X_train type: {type(X_train)}, shape: {getattr(X_train, 'shape', None)}")
print(f"{city_name} - y_train type: {type(y_train)}, shape: {getattr(y_train, 'shape', None)}")
print(f"{city_name} - X_val type: {type(X_val)}, shape: {getattr(X_val, 'shape', None)}")
print(f"{city_name} - y_val type: {type(y_val)}, shape: {getattr(y_val, 'shape', None)}")

Rio Branco - X_train type: <class 'numpy.ndarray'>, shape: (1136, 12, 9)
Rio Branco - y_train type: <class 'numpy.ndarray'>, shape: (1136,)
Rio Branco - X_val type: <class 'numpy.ndarray'>, shape: (0,)
Rio Branco - y_val type: <class 'numpy.ndarray'>, shape: (0,)


In [ ]:

# Batch GRU Profundo (Stacked) para Todas as Capitais Brasileiras
import sys
import os

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"Project root: {project_root}")

# Add the project root to sys.path (not the src directory)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.preprocessing import load_city_data, prepare_data_for_model, filter_city, clean_timeseries
from src.models import build_stacked_gru
from src.train import train_model, evaluate_model, generate_forecasts, save_predictions, save_metrics
from src.utils import plot_forecast, plot_forecast_error, plot_training_history
from tqdm import tqdm

# Set results_dir to the root project results directory (notebooks/../results/...)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
results_dir = os.path.join(project_root, 'results', 'dl_batch_state_capitals_morb_resp')
os.makedirs(results_dir, exist_ok=True)

np.random.seed(42)



# Lista de capitais e seus códigos IBGE
capitals = [
    ("Rio Branco", 1200401), ("Maceió", 2704302), ("Macapá", 1600303), ("Manaus", 1302603),
    ("Salvador", 2927408), ("Fortaleza", 2304400), ("Brasília", 5300108), ("Vitória", 3205309),
    ("Goiânia", 5208707), ("São Luís", 2111300), ("Cuiabá", 5103403), ("Campo Grande", 5002704),
    ("Belo Horizonte", 3106200), ("Belém", 1501402), ("João Pessoa", 2507507), ("Curitiba", 4106902),
    ("Recife", 2611606), ("Teresina", 2211001), ("Rio de Janeiro", 3304557), ("Natal", 2408102),
    ("Porto Alegre", 4314902), ("Porto Velho", 1100205), ("Boa Vista", 1400100), ("Florianópolis", 4205407),
    ("São Paulo", 3550308), ("Aracaju", 2800308), ("Palmas", 1721000)
]



# Parâmetros do modelo e dados
model_params = {
    'sequence_length': 12,
    'forecast_horizon': 4,
    'normalization': 'zscore',
    'val_size': None
}
target_column = 'target'
data_path = '../data/df_base_morb_resp.csv'
df = load_city_data(data_path)



def reduce_to_1d(arr):
    arr = np.asarray(arr)
    if arr.ndim == 1:
        return arr
    if arr.ndim == 2:
        if arr.shape[1] == 1:
            return arr.ravel()
        else:
            return arr.sum(axis=1)
    raise ValueError(f"Unexpected array shape: {arr.shape}")



for city_name, cd_mun in tqdm(capitals, desc='Capitais'):
    print(f"\n{'='*40}\n{city_name} ({cd_mun})")
    df_city = filter_city(df, cd_mun=cd_mun)
    # --- Robust interpolation for missing target values ---
    if df_city[target_column].isna().all():
        print(f"SKIP: {city_name} ({cd_mun}) - All target values are NaN. Skipping this city.")
        with open(os.path.join(results_dir, "skipped_cities_due_to_nan.txt"), "a", encoding="utf-8") as f:
            f.write(f"{city_name} ({cd_mun}): All target values are NaN\n")
        continue
    n_missing_before = df_city[target_column].isna().sum()
    if n_missing_before > 0:
        df_city[target_column] = df_city[target_column].interpolate(method='linear', limit_direction='both')
        # Fill any remaining NaNs at the edges
        df_city[target_column] = df_city[target_column].fillna(method='ffill').fillna(method='bfill')
        n_missing_after = df_city[target_column].isna().sum()
        print(f"{city_name}: Imputed {n_missing_before - n_missing_after} missing target values (linear+ffill+bfill). Remaining NaNs: {n_missing_after}")
    print(f"Shape após limpeza: {df_city.shape}")

    # --- Pipeline idêntico ao notebook individual ---
    data_dict = prepare_data_for_model(
        df=df_city,
        target_column=target_column,
        sequence_length=model_params['sequence_length'],
        forecast_horizon=model_params['forecast_horizon'],
        normalization=model_params['normalization'],
        val_size=model_params['val_size']
    )

    X_train = data_dict['X_train']
    y_train = data_dict['y_train']
    X_val = data_dict.get('X_val', None)
    y_val = data_dict.get('y_val', None)
    X_test = data_dict['X_test']
    y_test = data_dict['y_test']
    test_df = data_dict['test_df']
    scaler = data_dict.get('scaler')
    feature_columns = data_dict.get('feature_columns', None)

    print(f"X_train shape: {X_train.shape}")
    print(f"feature_columns: {feature_columns}")

    # --- NaN check and handling ---
    def has_nan(*arrays):
        return any(np.isnan(arr).any() for arr in arrays if arr is not None)

    nan_found = has_nan(X_train, y_train, X_val, y_val, X_test, y_test)
    if nan_found:
        print(f"WARNING: NaNs detected in data splits for {city_name}. Skipping this city.")
        with open(os.path.join(results_dir, "skipped_cities_due_to_nan.txt"), "a", encoding="utf-8") as f:
            f.write(f"{city_name} ({cd_mun}): NaNs in data splits\n")
        continue

    input_shape = X_train.shape[1:]
    model = build_stacked_gru(input_shape=input_shape, loss='huber')

    history = train_model(
        model=model,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        batch_size=32,
        epochs=400,
        patience=40,
        verbose=1
    )

    # --- Avaliação e denormalização ---
    y_pred = generate_forecasts(model, X_test)
    y_test_1d = reduce_to_1d(y_test)
    y_pred_1d = reduce_to_1d(y_pred)
    if scaler is not None and hasattr(scaler, 'scale_') and hasattr(scaler, 'mean_'):
        if scaler.scale_.shape[0] > 1:
            y_test_1d = y_test_1d * scaler.scale_[0] + scaler.mean_[0]
            y_pred_1d = y_pred_1d * scaler.scale_[0] + scaler.mean_[0]
        else:
            y_test_1d = scaler.inverse_transform(y_test_1d.reshape(-1, 1)).flatten()
            y_pred_1d = scaler.inverse_transform(y_pred_1d.reshape(-1, 1)).flatten()

    # --- Robust NaN check before evaluation ---
    if np.isnan(y_test_1d).any() or np.isnan(y_pred_1d).any():
        print(f"[SKIP] {city_name}: NaNs found in predictions or targets. Skipping evaluation and saving.")
        with open(os.path.join(results_dir, 'skipped_cities_due_to_nan.txt'), 'a', encoding='utf-8') as f:
            f.write(f"{city_name} ({cd_mun}): NaNs in y_test_1d or y_pred_1d\n")
        continue

    test_dates = test_df['week'].values[-len(y_test_1d):] if 'week' in test_df.columns else np.arange(len(y_test_1d))

    metrics = evaluate_model(
        model=model,
        X_test=X_test,
        y_test=y_test,
        scaler=scaler
    )

    print("Métricas de Avaliação:")
    print(f"MAE: {metrics['mae']:.4f}")
    print(f"RMSE: {metrics['rmse']:.4f}")
    print(f"R²: {metrics['r2']:.4f}")

    # Salvar previsões
    preds_file = save_predictions(
        y_true=y_test_1d,
        y_pred=y_pred_1d,
        dates=test_dates,
        city_name=city_name,
        model_name='gru_stacked',
        output_dir=results_dir
    )
    print(f"Previsões salvas em: {preds_file}")

    # Salvar métricas
    metrics_file = save_metrics(
        metrics=metrics,
        city_name=city_name,
        model_name='gru_stacked',
        output_dir=results_dir,
        params=model_params
    )
    print(f"Métricas salvas em: {metrics_file}")

    # Salvar histórico de treinamento
    if history is not None:
        fig = plot_training_history(history)
        fig.savefig(os.path.join(results_dir, f'{city_name}_gru_stacked_training_history.png'))


Project root: c:\Users\pedro\OneDrive - Unesp\Documentos\GitHub\cities-models\cities-models


NameError: name '__file__' is not defined

All predictions, metrics, and training history plots are saved in the results/dl_batch_state_capitals directory, one file per capital. You can rerun this notebook for other targets or model configs as needed.